<h2>Data Manipulation</h2>

In order to start the data analysis, we will need to import a variety of packages.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt
import math
import statsmodels.api as sm
from sklearn.preprocessing import scale
from sklearn.decomposition import PCA
from wordcloud import WordCloud, STOPWORDS

We will read in the data, which was downloaded from the Airbnb website and look at its info.

In [2]:
# Read in data
dat = pd.read_csv('datas/listings.csv')
dat.info()

FileNotFoundError: [Errno 2] No such file or directory: 'datas/listings.csv'

We can see that there are 74 columns, and 6,366 observations, however some values are missing in various columns.

In [ ]:
test = dat['calendar_last_scraped'] == dat['last_scraped']

In [ ]:
test.unique()

<h2>Data Cleansing</h2>

<h4>Drop off irrelevant columns and drop off reasons :</h4>

* URLs will not be useful
* Empty columns: **'neighborhood_group_cleansed'**, **'bathrooms'**, **'calendar_updated'**
* **'neighborhood'** column only has blank values or **'Chicago, Illinois, United States'** value, making it useless
* Drop **'host_listings_count'** and **'host_total_listings_count'**, use the calculated_host_listings columns, because values are the same.
* **'scrape_id'** is all the same value and not useful for our needs
* **'calendar_last_scraped'** can be dropped (same value as **'last_scraped'**)

In [ ]:
dat = dat.drop(['scrape_id','listing_url','host_url','host_thumbnail_url','host_picture_url','picture_url',
               'neighbourhood_group_cleansed','bathrooms','calendar_updated','neighbourhood',
               'calendar_last_scraped','host_listings_count','host_total_listings_count'], axis = 1)

<h4>Bathrooms Column:</h4>

In [ ]:
dat = dat.rename(columns={'id':'property_id', 'bathrooms_text':'bathrooms'})

In [ ]:
dat['bathrooms'].unique()

Looking at the 'bathrooms_text' column, we see that it is not very usable in its current state.  We will split the bathroom text column into two: one containing a float variable for the number of bathrooms, and the other an additional descriptor of the bathroom (shared/private).

In [ ]:
# First let us make all text lowercase to simplify string manipulation
# dat['bathrooms'] = dat['bathrooms'].str.lower()

# Next we must convert any text 'half' to 0.5 so it is included in the subsequent number extraction
#dat['bathrooms'] = dat['bathrooms'].str.replace(r'(half)+','0.5', regex = True)

# Then extract the numbers into the new 'bathrooms' float32 data type column
#dat['bathrooms'] = dat['bathrooms'].str.extract(r'(\d+\.?\d*)', expand = True).astype(np.float32)

# This leaves us with only float and NaN values
#dat['bathrooms'].unique()
# First let us split the text by white space
bath = dat['bathrooms'].str.split(' ', expand = True)
bath

In [ ]:
# then drop column 1 and 2, keep column 0
bath = bath.drop([1,2], axis=1)
dat['bathrooms'] = bath[0].replace(['Private', 'Shared', 'Half-bath'], '0.5')
dat['bathrooms'] = dat['bathrooms'].astype('float')
dat['bathrooms'].unique()

Above are the unique values left for the float variable in the column 'bathrooms'.

In [ ]:
dat['bathrooms'].value_counts()

This leaves us with only 1,587 observations with one of the bathroom descriptors, 'shared' or 'private'; the rest are missing values since the original data did not contain text for them.

<h4>DateTime Columns:</h4>

Next, let's convert the datetime columns into the proper datatype.

In [ ]:
# Convert dates to datetime data type
for x in ['last_scraped', 'host_since', 'first_review', 'last_review']:
    dat[x] = pd.to_datetime(dat[x])

**Create a new column to get how long the host exist**

In [ ]:
dat['host_period'] = dat['last_scraped'] - dat['host_since']

<h4>Rate Columns:</h4>

In [ ]:
dat['host_response_rate'].sample(5)

We can see that we will need to convert the percentage columns ('host_response_rate' and 'host_acceptance_rate') into float variables.

In [ ]:
# Convert host response rate and acceptance rate columns into float

dat['host_response_rate'] = dat['host_response_rate'].str.replace(r'(\D)','', regex = True).astype(np.float32)/100
dat['host_acceptance_rate'] = dat['host_acceptance_rate'].str.replace(r'(\D)','', regex = True).astype(np.float32)/100
dat['host_response_rate'].sample(5)

<h4>Boolean Columns:</h4>

In [ ]:
dat['host_is_superhost'].unique()

We will convert the 't' and 'f' values to binary float values for later analysis, where 1 will mean "True".

In [ ]:
# Map superhost column to boolean values
def repl_f_t(l):
    l = l.replace('f', 0)
    l = l.replace('t', 1);
    return l
dat['host_is_superhost'] = repl_f_t(dat['host_is_superhost']).astype('float');
dat['host_identity_verified'] = repl_f_t(dat['host_identity_verified']).astype('float')
dat['instant_bookable'] = repl_f_t(dat['instant_bookable']).astype('float')
dat['host_has_profile_pic'] = repl_f_t(dat['host_has_profile_pic']).astype('float')
dat['has_availability'] = repl_f_t(dat['has_availability']).astype('float')

#dat['host_is_superhost'] = dat['host_is_superhost'].map({'t':1,'f':0}).astype('float')
#dat['host_has_profile_pic'] = dat['host_has_profile_pic'].map({'t':1,'f':0}).astype('float')
#dat['host_identity_verified'] = dat['host_identity_verified'].map({'t':1,'f':0}).astype('float')
#dat['has_availability'] = dat['has_availability'].map({'t':1,'f':0}).astype('float')
#dat['instant_bookable'] = dat['instant_bookable'].map({'t':1,'f':0}).astype('float')

In [ ]:
dat['host_is_superhost']

<h4>Price Column:</h4>

Convert the price column to a float data type.

In [ ]:
dat['price'] = dat['price'].str.extract(r'(\d+\.\d+)').astype(np.float64)

In [ ]:
dat['price'].sample(5)

In [ ]:
dat['license'].sample(20)

Convert license to binary value: 1 = host have license number, 0 = host do not have license number

In [ ]:
#dat = dat.drop(['license'], axis=1)
dat['have_license'] = dat['license'].isnull()
dat['have_license'] = dat['have_license'].map({False:1, True:0})
dat

**Remove duplicate rows from dataset**

In [ ]:
dat = dat.drop_duplicates()

In [ ]:
dat = dat.loc[:,~dat.columns.duplicated()]
dat.head()

**Dummy Variable**

In [ ]:
x = pd.get_dummies(dat['host_is_superhost'])
dat = pd.concat([dat,x], axis=1)
dat = dat.rename(columns={0.0:'host_is_superhost_f', 1.0:'host_is_superhost_t'})

In [ ]:
x1 = pd.get_dummies(dat['host_identity_verified'])
dat = pd.concat([dat,x1], axis=1)
dat = dat.rename(columns={0.0:'host_identity_verified_f', 1.0:'host_identity_verified_t'})

In [ ]:
x2 = pd.get_dummies(dat['instant_bookable'])
dat = pd.concat([dat,x2], axis=1)
dat = dat.rename(columns={0.0:'instant_bookable_f', 1.0:'instant_bookable_t'})

In [ ]:
x2 = pd.get_dummies(dat['host_has_profile_pic'])
dat = pd.concat([dat,x2], axis=1)
dat = dat.rename(columns={0.0:'host_has_profile_pic_f', 1.0:'host_has_profile_pic_t'})

In [ ]:
x2 = pd.get_dummies(dat['has_availability'])
dat = pd.concat([dat,x2], axis=1)
dat = dat.rename(columns={0.0:'has_availability_f', 1.0:'has_availability_t'})

In [ ]:
dat.drop(['host_is_superhost_f', 'host_identity_verified_f', 'instant_bookable_f', 'host_has_profile_pic_f', 'has_availability_f'],axis=1).head()

**Missing value**

In [ ]:
pd.set_option('display.max_rows',100)
dat.isnull().sum()

For columns: **'host_response_time'**, convert to binary value and create a new column represent if a host respose in a day.

In [ ]:
dat['host_response_time'].unique()

In [ ]:
#Categorical within an hour,within a few hours, within a day to 1(true), NaN,a few days or more to 0(false) 
dat['host_response_inADay'] = dat.host_response_time.map({'within an hour': 1, 
                                                        'within a few hours': 1, 
                                                        'within a day':1, 
                                                        'a few days or more':0, 
                                                        np.nan:0})
dat['host_response_inADay'].unique()

For columns: **description**, **neighborhood_overview**, **host_location**, **host_about**, **host_neighbourhood**
     using 'Unknown' to fill the missing value, because these columns does not have direct effect on project topic.

In [ ]:
dat['description'].fillna(value='Unknown', inplace=True)

dat['neighborhood_overview'].fillna(value='Unknown', inplace=True)

dat['host_location'].fillna(value='Unknown', inplace=True)

dat['host_about'].fillna(value='Unknown', inplace=True)

dat['host_neighbourhood'].fillna(value='Unknown', inplace=True)

For columns: **host_name**, **host_since**, **host_has_profile_pic**, **host_identity_verified**, it is easy to see below these 10 rows contains many NaN value, so drop directly.

In [ ]:
dat[dat['host_name'].isnull()]

In [ ]:
dat.drop([dat.index[690], dat.index[744], dat.index[1541], dat.index[2705], dat.index[3189], dat.index[3240], dat.index[3630],
                dat.index[3965], dat.index[4671], dat.index[5755]],inplace = True)
dat.reset_index(drop=True, inplace=True)
dat[dat['host_name'].isnull()]

For columns: **bedrooms** and **beds**, using mode to fill missing value, because fill with natural value to make the result less biased. 

In [ ]:
dat['bedrooms'].mode()

In [ ]:
dat['bedrooms'].fillna(value=1.0, inplace=True)

In [ ]:
dat['bedrooms'].isnull().sum()

In [ ]:
dat['beds'].mode()

In [ ]:
dat['beds'].fillna(value=1.0, inplace=True)

In [ ]:
dat['bedrooms'].isnull().sum()

For columns: **first_review**, **last_review**, **review_scores_rating**, **review_scores_accuracy**, **review_scores_cleanliness**, **review_scores_checkin**, **review_scores_communication**, **review_scores_location**, **review_scores_value**, **reviews_per_month**, most missing value caused by 'number_of_reviews'==0. So, fill those missing value with 0.0. Later review analysis process will exclude these rows since no number of reviews.

In [ ]:
pd.set_option('display.max_columns',100)
dat[dat['number_of_reviews']==0]

In [ ]:
# first_review, last_review, will not be filled since no value suitable
dat['review_scores_rating'].fillna(value=0.0, inplace=True)

dat['review_scores_accuracy'].fillna(value=0.0, inplace=True)

dat['review_scores_cleanliness'].fillna(value=0.0, inplace=True)

dat['review_scores_checkin'].fillna(value=0.0, inplace=True)

dat['review_scores_communication'].fillna(value=0.0, inplace=True)

dat['review_scores_location'].fillna(value=0.0, inplace=True)

dat['review_scores_value'].fillna(value=0.0, inplace=True)

dat['reviews_per_month'].fillna(value=0.0, inplace=True)

In [ ]:
dat.isnull().sum()

<h2>Data Exploration and Visualization</h2>

Let us explore the data in the dataset.

<h4>Unique Hosts</h4>

First, how many unique hosts are there?

In [ ]:
dat['host_id'].nunique()

In [ ]:
listings_by_host = dat['host_id'].value_counts()
listings_by_host.describe()

In [ ]:
listings_by_host[listings_by_host > 1].count()

Here we can see that out of the 3,371 unique hosts, 799 have more than one listing in the Chicago area.  Interestingly, there is one host id with 260 listings.

Let's graph this data to see the distribution of hosts with differing numbers of listings.

In [ ]:
plt.figure(figsize=(18,6))
p = sb.countplot(x=listings_by_host, order=sorted(listings_by_host.unique()))
p.set_xticklabels(labels=p.get_xticklabels(),rotation=90)
p.bar_label(p.containers[0])
plt.xlabel('Number of Listings by Host')
plt.show()

It is obvious that a vast majority of hosts have only one listing in the Chicago area.

<h4>Host Response Time</h4>


In [ ]:
dat['host_response_time'].unique()

In [ ]:
plt.figure(figsize=(6,6))
p = sb.countplot(x='host_response_time',data = dat)
p.set_xticklabels(labels=p.get_xticklabels(),rotation=45)
p.bar_label(p.containers[0])
plt.xlabel('Host Response Time')
plt.show()

Here we can see there are four categories for the response time.  Let's define a system for rating the response time by using floating numbers.  We will assign the values in hours and as follows:
 * 'within an hour' = 1 hour
 * 'within a few hours' = 5 hours
 * 'within a day' = 24 hours
 * 'a few days or more' = 48 hours

In [ ]:
dat['host_response_time_float'] = dat['host_response_time'].map({'within an hour':1,'within a few hours':5,
                                                           'within a day':24,'a few days or more':48}).astype(np.float32)
dat['host_response_time_float'].unique()

<h4>Host Verifications</h4>

In order to simplify future analysis, let us count the number of verifications the host has and list this in a new column.

In [ ]:
dat['host_verifications'].sample(5)

We can see that the verifications are separated by a comma, so we will use this to count the number of verifications each host has.

In [ ]:
dat['no_of_verif'] = dat['host_verifications'].str.count(r',') + 1
dat.loc[:, ['host_verifications','no_of_verif']].head()

In order to catch any observations where hosts have no verifications, we will set the number of verifications to zero where the host_verifications = 'None'.  This is important since the code above would have counted 'none' and an observation without a comma (i.e. only one verification) as 1.

In [ ]:
dat['no_of_verif'] = np.where(dat['host_verifications'] == 'None', 0, dat['no_of_verif'])

In [ ]:
dat['no_of_verif'].describe()

On average, hosts have about 5-6 different identity verifications. 

In [ ]:
plt.figure(figsize=(18,6))
p = sb.countplot(x='no_of_verif',data = dat)
p.bar_label(p.containers[0])
plt.xlabel('Nomber of Identity Verifications by Host')
plt.show()

<h4>Amenities</h4>

Repeat the same process for number of amenities listed.

In [ ]:
dat['no_of_amen'] = dat['amenities'].str.count(r',') + 1
dat.loc[:, ['amenities','no_of_amen']].head()

In [ ]:
dat['no_of_amen'].describe()

In [ ]:
plt.figure(figsize=(18,6))
p = sb.countplot(x='no_of_amen',data = dat)
p.set_xticklabels(labels=p.get_xticklabels(),rotation=90)
plt.xlabel('Number of Amenities Listed by the Host')
plt.show()

<h4>Distance from Center of Chicago</h4>

Let's calculate the distance of the listing from the center of the city of Chicago.  We will use the following coordinates: 41.8781° N, 87.6298° W (source needed?).

We will use the Haversine formula to calculate the distance in miles.  In order to do so, we will first define a function to perform the calculation.

In [ ]:
# Coordinates for center of Chicago in degrees
lat1 = 41.8781
long1 = -87.6298

# Constant, radius of the Earth in miles
r = 3958.8

# Define a function to calculate the distance
def haversine(lat2, long2):
    # First convert degrees into radians:
    rlat1 = lat1 * (math.pi / 180)
    rlat2 = lat2 * (math.pi / 180)
    rlong1 = long1 * (math.pi / 180)
    rlong2 = long2 * (math.pi / 180)
    
    # Calculate the differnce between the latitudes and longitudes
    dlat = rlat1 - rlat2
    dlong = rlong1 - rlong2
    
    # Use the Haversine formula (broken into 3 terms for simplification here)
    a = (math.sin(dlat / 2) ** 2)         # First term
    b = math.cos(rlat1) * math.cos(rlat2) # Second term
    c = (math.sin(dlong / 2) ** 2)        # Third term
    e = math.sqrt(a + b * c)
    d = 2 * r * e                         # where r is the radius of the Earth
    return d

Then, apply the formula to each observation in the data set, returning the answer in a new column for the distance from the center of the city.

In [ ]:
dat['d_center'] = dat.apply(
    lambda row: haversine(row['latitude'], row['longitude']),
    axis=1)
dat['d_center'].sample(5)

In [ ]:
dat['d_center'].describe()

In order to visualize the distances, let us plot them against the listing price values.

In [ ]:
plt.figure(figsize=(12,8))
sb.regplot(x="d_center", y="price", 
            line_kws={"color":"r","alpha":0.5,"lw":3}, data=dat)
plt.xlabel('Distance from center of city (miles)')
plt.ylabel('Price ($)')
plt.show()

On first look, it appears that the listings further from the city are less valuable on average.

<h4>Property and Room Types</h4>

In [ ]:
plt.figure(figsize=(18,6))
p = sb.countplot(x='property_type',data = dat)
p.set_xticklabels(labels=p.get_xticklabels(),rotation=90)
plt.xlabel('Number of Amenities Listed by the Host')
plt.show()

In [ ]:
plt.figure(figsize=(12,6))
p = sb.countplot(x='room_type',data = dat)
plt.xlabel('Room Type')
plt.show()

<h2>Dimension Reduction</h2>



In [ ]:
dat.info()

In order to complete a Principal Component Analysis, we need to only select the numeric values and drop values with NaNs.

In [ ]:
# Select only numeric values
dat_pre_norm = dat.loc[:,['host_response_rate','host_acceptance_rate','host_is_superhost',
                         'host_total_listings_count','host_has_profile_pic',
                          'host_identity_verified','latitude','longitude','accommodates','bedrooms','beds',
                         'price','minimum_minimum_nights','maximum_minimum_nights',
                         'minimum_maximum_nights','maximum_maximum_nights','minimum_nights_avg_ntm',
                         'maximum_nights_avg_ntm','number_of_reviews','number_of_reviews_ltm','number_of_reviews_l30d',
                         'review_scores_rating','review_scores_accuracy','review_scores_cleanliness',
                         'review_scores_checkin','review_scores_communication','review_scores_location',
                         'review_scores_value','reviews_per_month']]
dat_pre_norm = dat_pre_norm.dropna()

In [ ]:
dat_norm = scale(dat_pre_norm)

For the fist PCA, we will include all of the variables (i.e. columns from the selection above).

In [ ]:
pca1 = PCA(n_components=len(dat_pre_norm.columns))

pca1.fit(dat_norm)

In [ ]:
var = pca1.explained_variance_ratio_
var1 = np.cumsum(np.round(pca1.explained_variance_ratio_, decimals=4)*100)
print(var1)

In [ ]:
var1 = pd.DataFrame(var1, index=np.arange(1,int(len(dat_pre_norm.columns))+1))
plt.plot(var1,color='blue')
plt.title('Scree Plot')
plt.xlabel('Number of Principal Components')
plt.ylabel('Cumulative Variance Explained')
plt.show()

Taking the first 18 principal components will correspond to 90% of the variance explained.

In [ ]:
pca18 = PCA(n_components=18)
pca18.fit(dat_norm)
data_pca18 = pca18.transform(dat_norm)

# Convert the numpy array to pandas DataFrame
data_pca18 = pd.DataFrame(data_pca18)
data_pca18.columns = ["PC"+str(i) for i in range(1,19)]

In [ ]:
data_pca18.corr()

From the above correlation table, we can find that the pairwise correlations between two components are close to zeros. This means that all these components are orthogonal (not correlated). There is no multicollinearity among principal components.

# Word Cloud

In [ ]:
dat.head()

## 1. Property Description Word Cloud

In [ ]:
dat.head()

In [3]:
stopwords = STOPWORDS
def Mywordcloud(data, title = None):   
    wc = WordCloud(
    background_color = "white",
    stopwords = stopwords,
    height = 600,
    width = 400
    ).generate(str(data))
    fig = plt.figure(1, figsize=(10, 10))
    plt.axis('off')
    if title:
        fig.suptitle(title, fontsize=20)
        fig.subplots_adjust(top=2.3)
    plt.imshow(wc)
    plt.show()

In [4]:
Mywordcloud(dat['description'].dropna())

NameError: name 'dat' is not defined

In [ ]:
Mywordcloud(dat['neighborhood_overview'].dropna())

In [ ]:
Mywordcloud(dat['host_about'].dropna())